In [18]:
%load_ext autoreload
%autoreload 2

import tnetwork as tn
import networkx as nx
import seaborn as sns
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
my_d_graph = tn.DynGraphSN()

In [12]:
my_d_graph.add_node_presence("a",1)
my_d_graph.add_nodes_presence_from(["a","b","c"],2)
my_d_graph.add_nodes_presence_from("d",[2,3])

my_d_graph.add_interaction("a","b",2)
my_d_graph.add_interactions_from(("b","d"),[2,3])

In [14]:
from bokeh.io import show, output_notebook, output_file

output_file("lines.html")
to_plot = tn.plot_longitudinal(my_d_graph,width=400,height=200,auto_show=False)
#output_notebook()
show(to_plot)

/Users/cazabetremy/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:2591: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


## Aggregating and slicing

In [70]:
from pytz import timezone
from datetime import timedelta, datetime
#Loading a predefined dataset, the sociopattern 2012 high school students dataset, in which, every 20s, face to face interaction between students are collected, for 7 days.
#timestamps are used to identify snapshots
SP = tn.DynGraphSN.graph_socioPatterns2012()
#Aggregate using a non-overlapping window of 30 minutes
SP30m = SP.aggregate_sliding_window(bin_size=60*30)
tn.plot_longitudinal(SP30m)
#tn.plot_longitudinal(SP30m,to_datetime=lambda x: datetime.fromtimestamp(x,timezone('Europe/Paris')))

/Users/cazabetremy/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:2591: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


Loading BokehJS ...

In [65]:
#Aggregate using overlapping windows of one day, every 1 hours
SP_1h_1d = SP.aggregate_sliding_window(bin_size=60*60*24,shift=60*60)

In [23]:
SP = tn.DynGraphSN.graph_socioPatterns2012()
SP_hour = SP.aggregate_sliding_window(60*60)


In [37]:
from datetime import timedelta
clusters=SP_hour.snapshots_timesteps()
level0 = [clusters[0:10],clusters[10:40],clusters[100:150]]
level1 = [clusters[0:50],clusters[50:200]]
all_levels = [level0,level1]
tn.plot_longitudinal_sn_clusters(SP_hour,all_levels,level=1,to_datetime=True,auto_show=True)

/Users/cazabetremy/anaconda3/lib/python3.6/site-packages/numpy/core/numeric.py:2591: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


Loading BokehJS ...

In [26]:
print(SP_hour.snapshots_timesteps())

[1353303380, 1353306980, 1353310580, 1353314180, 1353317780, 1353321380, 1353324980, 1353328580, 1353332180, 1353335780, 1353339380, 1353342980, 1353346580, 1353350180, 1353353780, 1353357380, 1353360980, 1353364580, 1353368180, 1353371780, 1353375380, 1353378980, 1353382580, 1353386180, 1353389780, 1353393380, 1353396980, 1353400580, 1353404180, 1353407780, 1353411380, 1353414980, 1353418580, 1353422180, 1353425780, 1353429380, 1353432980, 1353436580, 1353440180, 1353443780, 1353447380, 1353450980, 1353454580, 1353458180, 1353461780, 1353465380, 1353468980, 1353472580, 1353476180, 1353479780, 1353483380, 1353486980, 1353490580, 1353494180, 1353497780, 1353501380, 1353504980, 1353508580, 1353512180, 1353515780, 1353519380, 1353522980, 1353526580, 1353530180, 1353533780, 1353537380, 1353540980, 1353544580, 1353548180, 1353551780, 1353555380, 1353558980, 1353562580, 1353566180, 1353569780, 1353573380, 1353576980, 1353580580, 1353584180, 1353587780, 1353591380, 1353594980, 1353598580, 135